In [33]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
%matplotlib inline

V_MP = 0.03
V_REST = -0.065

class Hopfield:
    def __init__(self, size=64, iter=10):
        self.iter = iter
        self.size = size
        self.W = np.zeros((size ** 2, size ** 2))

    def train(self, X):
        n = self.size ** 2
        for x in X:  # (-1,64*64)
            x = np.reshape(x, (n, 1))
            xT = np.reshape(x, (1, n))
            self.W += x * xT / n
            self.W[np.diag_indices_from(self.W)] = 0

    def test_one_frame(self, x):
        n = self.size ** 2
        x = np.reshape(x, (n,))
        energy = []
        for iter in range(self.iter):
            h = np.zeros((n,))
            for i in range(n):
                i = np.random.randint(n)
                h[i] = self.W[i, :].dot(x)
                # print(i)
            
            x[h > 0] = 1
            x[h < 0] = -1
            energy.append(self.cal_energy(x))

        return np.resize(x, (self.size, self.size)), energy

    def cal_energy(self, x):
        n = self.size ** 2
        energy = np.sum(self.W.dot(x) * x)
        
        return -0.5 * energy

def show(x):
    img = np.where(x > 0, 255, 0).astype(np.uint8)
    cv.imshow("img", img)
    cv.waitKey(0)

class mIZH(Hopfield):
    def __init__(self, size=64, iter=5):
        super().__init__(size, iter)
        self.a1 = 0.04
        self.a2 = 5.0
        self.a3 = 140.0
        self.a4 = 1.0
        self.a5 = 1.0
        self.r = 0.02
            
    def test_one_frame(self, x):
        # n = self.size ** 2
        # x = np.reshape(x, (n,))
        # energy = []
        # for iter in range(self.iter):
        #     h = np.zeros((n,))
        #     for i in range(n):
        #         i = np.random.randint(n)
        #         h[i] = self.W[i, :].dot(x)
        #         # print(i)
        #     
        #     x[h > 0] = 1
        #     x[h < 0] = -1
        #     energy.append(self.cal_energy(x))
        # 
        # return np.resize(x, (self.size, self.size)), energy
        
        
        n = self.size ** 2
        x = np.reshape(x, (n,))
        il = np.zeros(n)
        il.astype(float)
        u = il.copy()
        
        for _ in range(self.iter):
            x = np.reshape(x,(n,1))  
            
            
            energy = []
            for i in range(len(u)):
                iext = 0;
                for j in range(self.size):
                    iext += self.W[j][i]*x[j]
                il[i] = il[i]+self.r*((self.a1*u[i]+self.a2) * u[i] + self.a3 - il[i])
                # u[i] = u[i] + (self.a1*u[i]+self.a2) * u[i]+self.a3-self.a4*il[i] + self.a5 * iext[i]
                u[i] = u[i] + (self.a1*u[i]+self.a2) * u[i]+self.a3-self.a4*il[i] + self.a5 * iext
                
                if abs(u[i]-V_REST) < 1e-5:
                    x[i] = x[i]
                else:
                    if u[i] > V_MP:
                        x[i] = 1;
                    else:
                        x[i] = -1;
            energy.append(self.cal_energy(x))
            # energy.append(0)
        # return X[-1]
        return np.resize(x, (self.size, self.size)), energy
        

这是一个markdown模板

In [ ]:
if __name__ == "__main__":

    img = cv.imread(r"D:\resource\mizh\kazuma.jpg", 0)
    size = 64
    img = cv.resize(img, (size, size))
    x = np.where(img > 255 / 2.5, 1, -1)
    x_masked = x.copy()
    x_masked[size // 2:, :] = -1
    img2  = cv.imread(r"D:\resource\mizh\fnn.jpg", 0)
    img2 =  cv.resize(img2, (size, size))
    x2 = np.where(img2 > 255 / 2.5, 1, -1)
    x_masked2 = x2.copy()
    x_masked2[size // 2:, :] = -1
    # show(x_masked)
    
    model =mIZH(size = size, iter=10)
    model.train([x])
    # y0 = model.train([x_masked2])
    # y0 = np.reshape(y0, (size,size))
    y, energy = model.test_one_frame(x_masked)
    show(y)
    plt.plot(energy, label='energy')
    plt.show()
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: RuntimeWarning: overflow encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: RuntimeWarning: overflow encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in double_scalars
